In [1]:
# region General Imports
import os
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

from model_optim.model_optimizer import ModelOptimizer

# Dataset
from custom_datasets.fatigue_mi import FatigueMI

%load_ext autoreload
%autoreload 3

/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-24 16:13:47.459649: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 16:13:47.460622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 16:13:47.467250: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 16:13:47.514230: I tensorflow/core/platform/cpu_feature_guard

To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [2]:
# np.load("./bkup/best_subjects_search_results/subject_6_shallow_conv_net_7b71b3b6-52f7-435c-be85-2cb99e793f98.npy", allow_pickle=True).item().user_attrs['trial_data']

prev_best_full = np.load("./_temp/[1]/292cbc92b8cf46da9986fe7d8447819f/model/study_best_trial.npy", allow_pickle=True).item()
# prev_best_full = np.load("./_temp/[2]/6405d11e654b42aca9df48458c67ecde/model/study_best_trial.npy", allow_pickle=True).item()
# prev_best_full = np.load("./_temp/[3]/9c1b753483db409a90eab7b7149b8af8/model/study_best_trial.npy", allow_pickle=True).item()
prev_best = prev_best_full.user_attrs['trial_data']

rpprint({
    "sfreq": prev_best_full.params["sfreq"],
    "batch_size": prev_best_full.params["batch_size"],
    "val_acc": np.max(prev_best["val_accuracy"]),
    "test_acc": prev_best["test_accuracy"],
    "channels": prev_best["channels_selected"],
})

{
│   'sfreq': 300,
│   'batch_size': 160,
│   'val_acc': 0.8333333134651184,
│   'test_acc': 0.8636363744735718,
│   'channels': array(['Fz', 'C4', 'Fp2', 'T5', 'O2', 'F7', 'F8', 'A2', 'T6', 'T4'],
│     dtype='<U3')
}

In [3]:
model_optimizer = ModelOptimizer(
    dataset=FatigueMI(),
    model_name="shallow_conv_net"
)

In [4]:
study = model_optimizer.search_best_model(
    subjects=[2],
    max_iter = 25,
    max_epochs=5,
    max_stag_count=10,
    rounds=1,
    replace_previous_study_for_subjects=True
)

  0%|          | 0/25 [00:00<?, ?it/s]

Adding metadata with 3 columns


2024-03-24 16:13:58.377350: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 16:13:58.705729: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 16:13:58.705812: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 16:13:58.709092: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 16:13:58.709811: I external/local_xla/xla/stream_executor

ValueError: No trials are completed yet.

In [ ]:
model_optimizer.get_study_metrics(study).sort_values(by="scores", ascending=True)
# model_optimizer.get_study_metrics(study).sort_values(by="test_acc", ascending=False)

,train_acc,test_acc,val_acc,train_val_acc_diff,train_loss,val_loss,train_val_loss_diff,test_loss,scores,channels_selected,sfreq,batch_size
9,0.867647,0.681818,0.722222,0.145425,10.666684,10.572948,0.093737,10.993606,0.077611,"[P3, F4, P4, Pz, O1, F7, F8, A2, T6]",300,160
7,0.838235,0.545455,0.666667,0.171569,13.874846,12.969428,0.905418,13.003958,0.111511,"[F3, F4, C4, P4, T3, O1, F8, T4]",128,32
15,1.000000,0.500000,0.611111,0.388889,10.528308,10.074472,0.453835,12.001544,0.151585,"[P3, C4, P4, O2, F7, F8, A2]",128,96
14,0.823529,0.500000,0.611111,0.212418,66.014801,62.986416,3.028385,63.090633,0.151735,"[P3, C3, F3, F4, C4, P4, Fp1, O2, T6, T4]",300,128
8,0.852941,0.590909,0.555556,0.297386,24.199411,22.264116,1.935295,22.316408,0.197981,"[F3, Cz, Pz, T3, O1, O2, F8, A2, T4]",128,256
18,0.985294,0.500000,0.444444,0.540850,30.577991,28.056805,2.521187,28.051445,0.309142,"[C3, P4, Pz, Fp1, Fp2, T3, F7, F8, A2, T4]",128,160
13,0.823529,0.636364,0.666667,0.156863,4.885457,4.687264,0.198193,6.606308,0.861561,"[C3, Fz, C4, Fp1, T3, T5, O1, O2, F7]",256,32
4,0.794118,0.545455,0.611111,0.183007,63.976913,58.828484,5.148430,79.104080,0.901635,"[C3, P4, T3, T5, O1, F7, F8, T6]",300,128
1,0.897059,0.545455,0.555556,0.341503,20.597481,19.734949,0.862532,23.798891,0.947981,"[C3, F3, P4, Fp1, Fp2, T3, O1, O2, T4]",300,96
11,0.985294,0.500000,0.555556,0.429739,15.905708,15.443455,0.462254,22.046785,0.948081,"[P3, C3, Fz, C4, Pz, Fp1, T3, O2, F7, T6, T4]",256,224


In [ ]:
rpprint({ k: v for k, v in study.best_trial.params.items() if not k.startswith("channels") })
rprint("test_accuracy =", study.best_trial.user_attrs["trial_data"]["test_accuracy"])
rprint("val_accuracy =", np.max(study.best_trial.user_attrs["trial_data"]["val_accuracy"]))
rprint("channels_selected =", study.best_trial.user_attrs["trial_data"]["channels_selected"])

{
│   'sfreq': 300,
│   'batch_size': 160,
│   'pool_size_d2': 15,
│   'strides_d2': 23,
│   'conv_filters_d2': 23,
│   'conv2d_1_units': 70,
│   'conv2d_2_units': 30,
│   'l2_reg_1': 0.29430426423889416,
│   'l2_reg_2': 0.2965775883025907,
│   'l2_reg_3': 0.05265734189484351,
│   'dropout_rate': 0.6
}

test_accuracy = 0.6818181872367859

val_accuracy = 0.7222222089767456

channels_selected = ['P3' 'F4' 'P4' 'Pz' 'O1' 'F7' 'F8' 'A2' 'T6']

In [20]:
# model_optimizer.clean_up(
#     action="remove_all_but_best_trial_data",
#     subjects=[1],
# )